In [5]:
import autorootcwd

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.feature_selection import RFECV, RFE
from xgboost import XGBClassifier

from src.data import artificial, spam
from src.evaluation import evaluate, search_knn, search_rf, search_xgboost

In [6]:
X_a, y_a = artificial.prepare_data()
X_spam, y_spam = spam.prepare_data()

In [7]:
c_s = [i for i in range(4, 51, 3)]

scores_knn = []
scores_rf = []
scoeres_xgb = []
params_knn_a = []
params_rf_a = []
params_xgb_a = []
features = []
n_features = []

In [ ]:
for c in c_s:
    classifier = XGBClassifier(learning_rate=0.01, max_depth=7, n_estimators=300)
    rfecv = RFE(estimator=classifier, step=3, n_features_to_select=c, verbose=1)
    rfecv.fit(X_a, y_a)

    selected_feature_indices = rfecv.get_support(indices=True)
    X_selected = X_a[:, selected_feature_indices]
    num_features = X_selected.shape[1]
    print(f"{num_features} features selected")

    params_knn, score_knn = search_knn(X_selected, y_a)
    params_rf, score_rf = search_rf(X_selected, y_a)
    params_xgb, score_xgb = search_xgboost(X_selected, y_a)

    scores = evaluate(X_selected, y_a, params_knn=params_knn, params_rf=params_rf, params_xgb=params_xgb, num_folds=10, scoring_coefficient=0.01)

    scores_knn.append(scores['KNN'][1])
    scores_rf.append(scores['RF'][1])
    scoeres_xgb.append(scores['XGB'][1])
    params_knn_a.append(params_knn)
    params_rf_a.append(params_rf)
    params_xgb_a.append(params_xgb)
    n_features.append(num_features)
    features.append(selected_feature_indices)

    print(f"KNN {scores['KNN'][1]} | RF {scores['RF'][1]} | XGB {scores['XGB'][1]}")

In [ ]:
df = pd.DataFrame(data=np.array([n_features, scores_knn, scores_rf, scoeres_xgb]).transpose(), columns=["features", "knn", "rf", "xgb"])